In [1]:
from preprocessing.src.preprocess.consolidation import Consolidate

In [2]:
conso = Consolidate(bucket_name='sqm-data-bucket', context='vertex')
wells_data = conso.consolide_data(station_type='wells')
conso.save_to_parquet(wells_data, file_name='wells_data')

meteo_data = conso.consolide_data(station_type='meteo')
conso.save_to_parquet(meteo_data, file_name='meteo_data')

2025-04-20 15:57:02,890 - INFO - Initializing Consolidate in context: 'vertex'
2025-04-20 15:57:02,894 - INFO - Consolidating wells data
2025-04-20 15:57:02,898 - INFO - Listing files in GCS directory: 'gs://sqm-data-bucket/raw-data/wells'
2025-04-20 15:57:02,981 - INFO - Found 29 files in 'gs://sqm-data-bucket/raw-data/wells'.
2025-04-20 15:57:02,982 - INFO - Processing 29 files for wells.
2025-04-20 15:57:02,983 - INFO - Reading Excel file: 'gs://sqm-data-bucket/raw-data/wells/1028.xlsx'
2025-04-20 15:57:03,811 - INFO - Successfully read raw-data/wells/1028.xlsx with 2598 rows and 6 columns.
2025-04-20 15:57:03,844 - INFO - Reading Excel file: 'gs://sqm-data-bucket/raw-data/wells/L1-17.xlsx'
2025-04-20 15:57:04,419 - INFO - Successfully read raw-data/wells/L1-17.xlsx with 2474 rows and 7 columns.
2025-04-20 15:57:04,464 - INFO - Reading Excel file: 'gs://sqm-data-bucket/raw-data/wells/L1-3.xlsx'
2025-04-20 15:57:04,858 - INFO - Successfully read raw-data/wells/L1-3.xlsx with 2058 row